This script compares various sentence similarity techniques

In [ ]:
# !pip install --upgrade dutch_text_analytics

In [ ]:
import pandas as pd
import numpy as np
import os
import dutch_text_analytics.textanalytics as ta
from dutch_text_analytics import demo_data

In [ ]:
print("initialising simple extractors")
spacy_extractor = ta.Embedder(method='spacy',suffix='token_based')
# doc2vec_extractor = ta.Embedder(method='doc2vec', model_path='models/doc2vec_model')
dberttf_extractor = ta.Embedder(method='TF', model_path='distilbert-base-uncased', suffix='dbert-tf')
mlbert_extractor = ta.Embedder(method='TF', model_path='distilbert-base-multilingual-cased', suffix='mlbert')
glove_extractor = ta.Embedder(method='sentTF', model_path='sentence-transformers/average_word_embeddings_glove.6B.300d', suffix='glove')
print("initialising transformer extractors")
bert_extractor = ta.Embedder(method='TF', model_path='bert-base-uncased', suffix='bert')
robbert_extractor = ta.Embedder(method='TF', model_path='pdelobelle/robbert-v2-dutch-base', suffix='robbert')
bertje_extractor = ta.Embedder(method='TF', model_path='GroNLP/bert-base-dutch-cased',suffix='bertje')
medrobertanl_extractor = ta.Embedder(method='TF', model_path='CLTL/MedRoBERTa.nl', suffix='medrobertanl')
print("initialising traditional extractors")
jaccard_extractor = ta.Embedder(method='jaccard')
wmd_extractor = ta.Embedder(method='wmd')
ld_extractor = ta.Embedder(method='ld')


In [ ]:
sentences = demo_data.sentences
references = demo_data.references

In [ ]:
text_public.sentences

In [ ]:

result_dict = {'sentence':[], 'reference':[], 'method':[], 'comparison_type':[], 'measure':[]}

chosen_extractors = [
                    spacy_extractor, 
                    doc2vec_extractor, 
                    dberttf_extractor,
                    mlbert_extractor, 
                    glove_extractor, 
                    bert_extractor, 
                    robbert_extractor, 
                    bertje_extractor,
                    medrobertanl_extractor,
                    jaccard_extractor, 
                    wmd_extractor, 
                    ld_extractor,
                     ]
for extractor in chosen_extractors:
  print('Model:',extractor.name)
  for sentence1 in sentences.keys():
      for sentence2 in references:
            measure = extractor.get_comparison(sentence1,sentence2)
            result_dict['sentence'].append(sentence1)
            result_dict['reference'].append(sentence2)
            result_dict['method'].append(extractor.name)
            result_dict['comparison_type'].append(extractor.comparison_type)
            result_dict['measure'].append(measure)
        
result = pd.DataFrame(result_dict)
print(result)    
result_file = 'outputs/compare_sentence_similarity_methods_fast.xlsx'
result.to_excel(result_file)



In [ ]:


all_measures = list(result.method.unique())+['ensemble']
all_measure_dict = {i:{'correct_match_count':0, 'partially_correct_match_count':0,'incorrect_match_count':0, 'correct_matches':{},'partially_correct_matches':{},'incorrect_matches':{}} for i in all_measures}
N = 2
for sen in result.sentence.unique():
    for method in all_measures:
        for type2 in result.comparison_type.unique():
            if method == 'ensemble':
                score_df = result[(result.sentence == sen)]
                type1 = score_df.comparison_type.unique()[0]
            else:
                score_df = result[(result.sentence == sen) & (result.method == method)]
                type1 = score_df.comparison_type.unique()[0]
            if type1 != type2:
                continue
            if type1 == "similarity":
                best_matches = score_df.sort_values(['measure'], ascending=False)[:N]['reference'].to_list()
                best_scores = score_df.sort_values(['measure'], ascending=False)[:N]['measure'].to_list()
            if type1 == "distance":
                best_matches = score_df.sort_values(['measure'], ascending=True)[:N]['reference'].to_list()
                best_scores = score_df.sort_values(['measure'], ascending=True)[:N]['measure'].to_list()

            best_match_stat = [f'Measure:{score:.2f} {match}' for (match,score) in zip(best_matches, best_scores)]
            if any(set(best_matches).intersection(set(sentences[sen][:1]))):
                all_measure_dict[method]["correct_match_count"] += 1
                all_measure_dict[method]["correct_matches"].update({sen:best_match_stat})
            elif any(set(best_matches).intersection(set(sentences[sen]))):
                all_measure_dict[method]["partially_correct_match_count"] += 1
                all_measure_dict[method]["partially_correct_matches"].update({sen:best_match_stat})
            else:
                # print('**',sen,method,type1,type2,best_matches,'**')
                all_measure_dict[method]["incorrect_match_count"] += 1
                all_measure_dict[method]["incorrect_matches"].update({sen:best_match_stat})

for k,v in all_measure_dict.items():
    print(k)
    for k_,v_ in v.items():
        print(k_)
        if type(v_) == dict:
            for k__,v__ in v_.items():
                print('\t',k__,':',)
                if type(v__) == list:
                    for i in v__:
                        print('\t\t:',i)
        else:
            print('\t',k_,':',v_)
    print("************************")
    print()


In [ ]:
print("Summary:")
summary_dict = {'method_name':[], 'correct_match_count':[], 'partially_correct_match_count':[], 'incorrect_match_count':[]}
for k,v in all_measure_dict.items():
    summary_dict['method_name'].append(k)
    for k_,v_ in v.items():
        if k_ in ['correct_match_count','partially_correct_match_count','incorrect_match_count',]:
            summary_dict[k_].append(v_)
summary_df = pd.DataFrame(summary_dict)
print(summary_df)
summary_file = 'outputs/compare_sentence_similarity_methods_fast_summary.xlsx'
summary_df.to_excel(summary_file)

In [ ]:
from textanalytics import Classifier
mdeberta_classifier = Classifier(method='TF', classes = list(references), model_path="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", suffix="mDeBERTa", multi_label=False)

analysis_dict = {'sentence':[],'gt':[], 'prediction':[]}
for sentence,gt_labels in sentences.items():
    gt_label = gt_labels[0]
    result = mdeberta_classifier.classify(sentence)
    predicted_label = result['labels'][np.argmax(result['scores'])]
    analysis_dict['sentence'].append(sentence)
    analysis_dict['gt'].append(gt_label) 
    analysis_dict['prediction'].append(predicted_label)   
analysis_df =  pd.DataFrame(analysis_dict)



In [ ]:
analysis_df